# **Project Pipeline**

| **Steps**                                              | **Script files**                          |
|-----------------------------------------------------------|-------------------------------------------|
| 1) Read and pre-process data                              | pre_processing.py                         |
| 2) Feature engineering                                    | feature_engineering.py                    |
| 3) Train models                                           | model_training.py, <br>tree_model_training.py |
| 4) Predict on test_features <br>and write submission file | final_predict.py                          |

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

import scripts.pre_processing as pp
import scripts.feature_engineering as fe
import scripts.model_evaluation as me
from scripts.model_training import Model
from scripts.tree_model_training import rf_model
from scripts.model_evaluation import regression_evaluation
import scripts.final_predict as fp
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
import sklearn.preprocessing import MinMaxScaler 




## Pre-process data

In [5]:
# Read in data files 
train_features = pd.read_csv('./data/dengue_features_train.csv')
train_target = pd.read_csv('./data/dengue_labels_train.csv')
test_features = pd.read_csv('./data/dengue_features_test.csv')

In [6]:
# Merge features and target data
data = pp.merge_data(train_features, train_target, test_features, inc_test=False)

# Run processing and split by city
train_iq = pp.pre_process(data, 'iq', remove_anomalies=True)
train_sj = pp.pre_process(data, 'sj', remove_anomalies=True)

# Run checks for missing values
assert train_iq.isnull().any().any() == False
assert train_sj.isnull().any().any() == False
print(f'train_iq shape: {train_iq.shape}') 
print(f'train_sj shape: {train_sj.shape}') 

train_iq shape: (520, 24)
train_sj shape: (928, 24)


In [ ]:
#plt.boxplot(train_sj['total_cases'])
d = train_iq.loc[train_iq['total_cases'] <80, :] 
plt.boxplot(d['total_cases'])


In [ ]:
# Run feature engineering 
train_iq = fe.feature_engineer_1(train_iq)
train_sj = fe.feature_engineer_1(train_sj)

In [ ]:
# Split into training and cross-validation sets
X_train_sj, y_train_sj, X_test_sj, y_test_sj = pp.train_cv_split(train_sj, city='sj')
X_train_iq, y_train_iq, X_test_iq, y_test_iq = pp.train_cv_split(train_iq, city='iq')

# Check compatible sizes for models:
assert len(X_train_sj) == len(y_train_sj)
assert len(X_test_sj) == len(y_test_sj)
assert len(X_train_iq) == len(y_train_iq)
assert len(X_test_iq) == len(y_test_iq)

In [16]:
# Train test split with sklearn 
tss = TimeSeriesSplit(n_splits = 3)

X_iq = train_iq.drop(labels=['total_cases'], axis=1)
y_iq = train_iq['total_cases']
X_sj = train_sj.drop(labels=['total_cases'], axis=1)
y_sj = train_sj['total_cases']

for train_index, test_index in tss.split(X_iq):
    X_train, X_test = X_iq.iloc[train_index, :], X_iq.iloc[test_index,:]
    y_train, y_test = y_iq.iloc[train_index], y_iq.iloc[test_index]
    print(X_train.shape, X_test.shape)

(130, 23) (130, 23)
(260, 23) (130, 23)
(390, 23) (130, 23)


## Train model

In [ ]:
# Baseline model predictions for San Jose (sj)
bl_pred_train = np.tile(np.mean(y_train_sj), len(y_train_sj))
bl_pred_test = np.tile(np.mean(y_test_sj), len(y_test_sj))
regression_evaluation(y_train_sj, y_test_sj, bl_pred_train, bl_pred_test)

In [ ]:
# Baseline model predictions for Iquitos (Iq)
bl_pred_train = np.tile(np.mean(y_train_iq), len(y_train_iq))
bl_pred_test = np.tile(np.mean(y_test_iq), len(y_test_iq))
regression_evaluation(y_train_iq, y_test_iq, bl_pred_train, bl_pred_test)

In [ ]:
# Tree model for IQ
rf_model(X_train_iq, y_train_iq, X_test_iq, y_test_iq)

In [ ]:
# Tree model for SJ 
rf_model(X_train_sj, y_train_sj, X_test_sj, y_test_sj)

In [3]:
# LTSM MODEL 
from sklearn.preprocessing import MinMaxScaler 
import tensorflow as tf
from tf.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
from tf.compat.v1.keras.layers import CuDNNLSTM
from tf.keras.models import Sequential


scaler = MinMaxScaler()
scaled_X = scaler.fit_transform(X_iq)

model = Sequential(name="LSTM-Model")
model.add(LSTM(units=3, input_shape=(2,10), return_sequences=False))
model.add(Dense(1, activation='Linear'))
model.complile(loss='mae', optimizer='adam')
model.fit(scaled_X, y_iq, epochs=10)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# LTSM MODEL 
import sklearn.preprocessing import MinMaxScaler 
import tensorflow as tf
from tf.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
from tf.compat.v1.keras.layers import CuDNNLSTM
from tf.keras.models import Sequential


scaler = MinMaxScaler()
scaled = scaler.fit_transform(X_iq)

model = Sequential(name="LSTM-Model")
model.add(LSTM(units=3, input_shape=(2,10), return_sequences=False)
model.add(Dense(1, activation='Linear'))
model.complile(loss='mae', optimizer='adam')
model.fit(X_iq, y_iq, epochs=10)

model = Sequential(name="LSTM-Model") # Model
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-LSTM-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(Y_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-LSTM-Decoder-Layer')) # Decoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x



...
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()


## Optimise models

## Predict on test data with chosen model and write file

In [ ]:
# Code to run pipeline on dataset including test_features, and then take only test_features to run the final model

In [ ]:
# Select data and chosen model and hyperparameters for final prediction

# Iquitos, iq
final_test_iq = train_iq.drop(['total_cases'], axis=1)
X_train_iq = X_train_iq
y_train_iq = y_train_iq
model_iq = 'RandomForestRegressor'
params_iq = {}

# San Jose, sj
final_test_sj = train_sj.drop(['total_cases'], axis=1)
X_train_sj = X_train_sj
y_train_sj = y_train_sj
model_sj = 'RandomForestRegressor'
params_sj = {}

In [ ]:
# Perform final predictions and reformat for submission
final_iq = fp.final_predict(final_test_iq, X_train_iq, y_train_iq, 
              city='iq', model=model_iq, params=params_iq)
final_sj = fp.final_predict(final_test_sj, X_train_sj, y_train_sj, 
              city='sj', model=model_sj, params=params_sj)

# Merge the two cities into one DataFrame and write to new csv file 
fp.write_submission(final_iq, final_sj) 

# 